In [ ]:
# -- Tensorflow -- #
import tensorflow as tf

from tensorflow.keras.layers import (
    Softmax,
    Dense,
    AdditiveAttention,
    MultiHeadAttention,
    Layer,
    LayerNormalization,
    Dropout,
    Embedding
)

from tensorflow.keras import (
    Sequential,
    Model
)


# Transformer Pipeline

In [ ]:
@dataclass
class TransformerGlobalConfig:
    d_model: int = 512
    max_seq_length: int = 200
    global_training: bool = True
    storage_path: str = '/results/'

    def load(self, config):
        set_attributes(self, config)


@dataclass
class BERTLayerConfig:
    num_attention_heads: int = 8
    num_encoder_layers: int = 12
    dff: int = 2048
    max_seq_len: int = 2048
    dropout_rate: float = 0.1
    load_model: bool = False
    save_model: bool = True
    training: bool = True

    def load(self, config):
        set_attributes(self, config)


@dataclass
class HitAnomalyLayerConfig:
    num_attention_heads: int = 12
    num_encoder_layers: int = 3
    dff: int = 2048
    max_seq_len: int = 2048
    dropout_rate: float = 0.1
    load_model: bool = False
    save_model: bool = True
    training: bool = True

    def load(self, config):
        set_attributes(self, config)


class TransformerConfig:
    def __init__(self):
        self._global = TransformerGlobalConfig()
        self.BERT = BERTLayerConfig()
        self.HitAnomaly = HitAnomalyLayerConfig()

    def load(self, path):
        try:
            with open(path) as f:
                transformer_config = yaml.load(f, Loader=yaml.FullLoader)
        except FileNotFoundError as e:
            logger.warning(e)
            return None

        self._global.load(transformer_config)
        self.BERT.load(transformer_config)
        self.HitAnomaly.load(transformer_config)

def set_attributes_from_object(self, *args):
    try:
        for obj in args:
            for attr_key, attr in obj.__dict__.items():
                setattr(self, attr_key, attr)
    except Exception as e:
        logger.warning(e)

## Metric Objects

### Loss Function

In [ ]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

### Accuracy Function

In [ ]:
def accuracy_function(real, pred):
    accuracies = tf.equal(real, tf.argmax(pred, axis=1))

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies) / tf.reduce_sum(mask)

### Custom Learning Rate Schedule

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model: int, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

## Pipeline Objects

### PositionalEncodingLayer

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_steps, max_dims, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        if max_dims % 2 == 1: max_dims += 1  # max_dims must be even
        p, i = np.meshgrid(np.arange(max_steps), np.arange(max_dims // 2))
        pos_emb = np.empty((1, max_steps, max_dims))
        pos_emb[0, :, ::2] = np.sin(p / 10000 ** (2 * i / max_dims)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10000 ** (2 * i / max_dims)).T
        self.positional_embedding = tf.constant(pos_emb.astype(self.dtype))

    def call(self, inputs):
        shape = tf.shape(inputs)
        return inputs + self.positional_embedding[:, :shape[-2], :shape[-1]]

### EncoderBlock

In [ ]:
class EncoderBlock(Layer):

    def __init__(
        self,
        d_model: int,
        num_heads: int,
        dff: int,
        rate=0.1):
        super(EncoderBlock, self).__init__()

        self.multi_headed_attention = MultiHeadAttention(num_heads=num_heads,
                                                         key_dim=d_model // num_heads,
                                                         dropout=0.1,
                                                         attention_axes=(1))

        self.feed_forward_network = Sequential([
            Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
            Dense(d_model, activation='relu')  # (batch_size, seq_len, d_model)
        ])

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, x, mask):
        # (1) - Attention Score
#         logger.info('MULTIHEADED ATTENTION')
#         logger.info(x.shape)
        attn_output, attn_weights = self.multi_headed_attention(
            x,
            x,
            return_attention_scores=True)  # (batch_size, input_seq_len, d_model)

        # (2) - Add & Normalize
        attn_output = self.dropout1(attn_output, training=True)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        # (3) - Feed Forward NN
        feed_forward_output = self.feed_forward_network(out1)  # (batch_size, input_seq_len, d_model)

        # (4) - Add & Normalize
        feed_forward_output = self.dropout2(feed_forward_output, training=True)
        out2 = self.layernorm2(out1 + feed_forward_output)  # (batch_size, input_seq_len, d_model)

        return tf.convert_to_tensor(out2), tf.convert_to_tensor(attn_weights)

### BERT

In [ ]:
class BERTLayer(Layer):
    def __init__(
        self,
        global_config: TransformerGlobalConfig,
        config: BERTLayerConfig):
        super(BERTLayer, self).__init__()

        set_attributes_from_object(
            self,
            global_config,
            config)

        self.bert_layer_blocks = [EncoderBlock(
            self.d_model,
            self.num_attention_heads,
            self.dff,
            rate=self.dropout_rate) for _ in range(self.num_encoder_layers)]

    def call(self, input_: tf.tuple, **kwargs):
        enc_input = input_[0]
#         logger.info('BERT LAYER')
#         logger.info(enc_input.shape)
        encoding_padding_mask = None
        # BERT for Log Sequence Embedding
        for layer_idx in range(self.num_encoder_layers):
#             logger.info('BERT LAYER LOOP')
#             logger.info(enc_input.shape)
            enc_output, attention = self.bert_layer_blocks[layer_idx](enc_input, encoding_padding_mask)
            bert_ret = tf.tuple(enc_output, attention)
        return bert_ret

### HitAnomaly

In [ ]:
class HitAnomalyLayer(Layer):
    def __init__(
        self,
        vocab_size: int,
        global_config: TransformerGlobalConfig,
        config: HitAnomalyLayerConfig):
        super(HitAnomalyLayer, self).__init__()

        self.vocab_size = vocab_size
        set_attributes_from_object(
            self,
            global_config,
            config)

        self.encoding_blocks = [EncoderBlock(
            self.d_model,
            self.num_attention_heads,
            self.dff,
            rate=self.dropout_rate
        ) for _ in range(self.num_encoder_layers)]

        self.hidden_layer_output = []

#     @tf.function(jit_compile=True)
    def call(self, input_: tf.tuple, **kwargs):
        enc_input = input_[0]
        encoding_padding_mask = None

        # Encoder Block Hidden Layers for Log Encoder
        # (batch_size, inp_seq_len, d_model), (batch_size, class, inp_seq_len, inp_seq_len)
        for layer_idx in range(self.num_encoder_layers - 1):
            enc_output, att = self.encoding_blocks[layer_idx](enc_input, encoding_padding_mask)
            self.hidden_layer_output.append(enc_output)

        fin_output = enc_output
        final_output = tf.reduce_mean(fin_output, axis=1)
        final_output = tf.expand_dims(final_output, axis=0)

        # Last Encoding Block for Log Sequence Representation
        out, att = self.encoding_blocks[self.num_encoder_layers - 1](final_output, encoding_padding_mask)
        self.hidden_layer_output.append(out)

        # Final Pooling Layer
        seq_representation = tf.reduce_mean(out, axis=1)

        return seq_representation, att

### Transformer

In [ ]:
class Transformer(Model):

    def __init__(
        self,
        tokenizer: PrimeTokenizer,
        config: TransformerConfig):
        super(Transformer, self).__init__()

        self.vocab_size = tokenizer.get_vocab_size()
        set_attributes_from_object(
            self,
            config._global)

        self.embedding = Embedding(
            self.vocab_size,
            self.d_model,
            input_length=self.max_seq_len)

        self.pos_encoding = PositionalEncoding(
            self.max_seq_len,
            self.d_model)

        self.bert_layer = BERTLayer(
            config._global,
            config.BERT)

        self.hitanomaly_layer = HitAnomalyLayer(
            self.vocab_size,
            config._global,
            config.HitAnomaly)

        #self.dropout = Dropout(rate)

#     @tf.function(jit_compile=True)
    def call(self, input_tuple: tf.tuple, **kwargs):
        log_batch = input_tuple[0]
#         logger.info('INITIAL')
#         logger.info(log_batch.shape)
        encoding_padding_mask = None # input_tuple[1]

        embedding_tensor = self.embedding(log_batch) # (batch_size, input_seq_len, d_model)
#         logger.info('POST EMBEDDING LAYER')
#         logger.info(embedding_tensor.shape)

        embedding_tensor = self.pos_encoding(embedding_tensor)
#         logger.info('POST POSITIONAL ENCODING')
#         logger.info(embedding_tensor.shape)
        #embedding_tensor = self.dropout(embedding_tensor, training=TRAINING)

        # BERT for Log Sequence Embedding
        bert_arg = tf.tuple(embedding_tensor, encoding_padding_mask)
        bert_ret = self.bert_layer(bert_arg)

        # Encoder Block Hidden Layers for Log Sequence Representation
#         seq_representation, att = self.hitanomaly_layer(tf.tuple(enc_output, encoding_padding_mask))


        return bert_ret

## Transformer Main

### Batch Processing

### Main (Initialization)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

# -- Transformer Model -- #
transformer_config_path = SOURCE + '/assets/notebooks/TransformerConfig.yaml'
transformer_config = TransformerConfig()
transformer_config.load(transformer_config_path)
# optimus_prime = Transformer(prime_tokenizer, transformer_config)

t_config = transformer_config._global

# -- Pipeline Info -- #
attns = []

# -- Data Batches -- #
# batched_dataset = process_all_batches(n_iter, log_labels, t_config.batch_size)

# -- Model Metrics -- #
learning_rate = CustomSchedule(t_config.d_model)
epoch_loss = Mean(name='train_loss')
epoch_accuracy = Mean(name='train_accuracy')
loss_object = SparseCategoricalCrossentropy(from_logits=True)
optimizer = Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# -- Classification Step Layers -- #
add_att_layer = AdditiveAttention()
softmax = Softmax()
s1 = Sequential([
    Dense(t_config.batch_size, activation=t_config.activation),
    Dense(4, activation=t_config.activation),
    Softmax()
])

# -- Checkpoints -- #
# checkpoint_path = SOURCE + "checkpoints/"
# checkpoint = Checkpoint(step=tf.Variable(1), transformer=optimus_prime, optimizer=optimizer)
# checkpoint_manager = CheckpointManager(checkpoint, checkpoint_path, max_to_keep=5)

# tf.debugging.set_log_device_placement(True)
writer = tf.summary.create_file_writer(SOURCE + t_config.logdir)

### TrainStep

In [ ]:
batch_size = transformer_config._global.batch_size
max_seq_len = transformer_config._global.max_seq_len

train_step_signature = [
    tf.TensorSpec(shape=([batch_size, max_seq_len]), dtype=tf.int32),
    tf.TensorSpec(shape=([batch_size]), dtype=tf.int8)
]

@tf.function(input_signature=train_step_signature,
             experimental_compile=True)
def train_step(log_batch: tf.Tensor, labels: tf.Tensor):

    transformer_input = tf.tuple([
        log_batch,  # <tf.Tensor: shape=(batch_size, max_seq_len), dtype=int32>
        labels  # <tf.Tensor: shape=(batch_size, num_classes), dtype=float32>
    ])

    with tf.GradientTape() as tape:
        transformer_ret = optimus_prime(transformer_input)
#         a_s = add_att_layer([Rs, Rs])
#         y = softmax(a_s * Rs)
#         print(a_s.shape)
        # y = Rs
#         loss = tf.py_function(loss_function, [labels, y], tf.float32)
#         pred = s1(y)
#         labels = tf.cast(labels, tf.int8)
    # Optimize the model
#     grads = tape.gradient(loss, optimus_prime.trainable_variables)
#     optimizer.apply_gradients(zip(grads, optimus_prime.trainable_variables))

#     acc = accuracy_function(labels, pred)

    # Tracking Progress
#     epoch_loss.update_state(loss)  # Adding Batch Loss
#     epoch_accuracy.update_state(acc)

    return transformer_ret

In [ ]:
train_step_signature

### Main (Training)

In [ ]:
attentions = []

for epoch in range(t_config.epoch):

    start = time.time()
    epoch_loss.reset_states()
    epoch_accuracy.reset_states()
    dataset_iter = iter(batched_dataset)

    t = tqdm(range(n_iter), desc="Epoch: {:03d}, Loss: {:.3f}, Accuracy: {:.3%}".format(0, 0, 0), position=0, leave=True)
    for _ in t:
        batch = next(dataset_iter)
        log_batch = batch[0]
        labels = batch[1]

        # Returns Eager Tensor for Predictions
#         tf.summary.trace_on()
#         tf.profiler.experimental.start(SOURCE + t_config.logdir)

#         with writer.as_default():
        transforer_ret = train_step(log_batch, labels)
        attentions.append(transformer_ret)
          # with tf.summary.record_if(True):

#             tf.summary.trace_export(
#               name = "training_trace",
#               step=0,
#               profiler_outdir=SOURCE + t_config.logdir
#             )

#         tf.profiler.experimental.stop()
#         tf.summary.trace_off()

#         checkpoint.step.assign_add(1)

#         if int(checkpoint.step) % 10 == 0:
#             save_path = checkpoint_manager.save()

        t.set_description(desc="Epoch: {:03d}, Loss: {:.3f}, Accuracy: {:.3%} ".format(epoch,
                                                                    epoch_loss.result(),
                                                                    epoch_accuracy.result()))
        t.refresh()

In [ ]:
joblib.dump(acc, SOURCE + '/results/attention.joblib')

In [ ]:
joblib.dump(Rs, SOURCE + '/results/Rs.joblib')